# EMT Model Evaluator
### Author: Joel Meoak
### This notebook is used to evaluate the trained models against the ember dataset and output standard ML metrics. While the standard metrics are important, it is also vital to avoid overfitting to the EMBER data as the final evaluation will be against malware 

In [2]:
import ember
from joblib import dump, load

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, f1_score, precision_score

## Import Testing Data

In [3]:
X_train, y_train, X_test, y_test = ember.read_vectorized_features(r"C:\Users\joelm\anaconda3\A_Working\ember2018\ember2018")
metadata_dataframe = ember.read_metadata(r"C:\Users\joelm\anaconda3\A_Working\ember2018\ember2018")

Changes X_test and y_test to pandas df and creates a new version of x that includes only dlls. Feature number 619 is number of exports and number of exports = 0 means that it is a exe, and exports > 0 indicates its a dll. 

In [4]:
X_test_pd = pd.DataFrame(data=X_test, copy=True)
y_test_pd = pd.DataFrame(data=y_test, copy=True)
mask = X_test_pd[619] != 0
X_test_dlls = X_test_pd[mask]

# Traditional Models

## Light BGM Model
This model was created by the EMBER Dataset Authors as a benchmark to compare against other models

The Light BGM model provides a likelihood score between 0 and 1. Using different rounding criteria will adjust the precision to recall balance. 

In [142]:
lgbm_model = load('lgbm.joblib')

In [143]:
lgbm_pred = lgbm_model.predict(X_test)

In [144]:
lgbm_pred_rounded = np.where(lgbm_pred >= 0.9, 1, 0)

In [145]:
print('LGBM Benchmark Total Test')
print("Accuracy Score:", accuracy_score(y_test, lgbm_pred_rounded))
print("Precision Score:", precision_score(y_test, lgbm_pred_rounded))
print("Recall Score:",recall_score(y_test, lgbm_pred_rounded))
print("F1 Score:",f1_score(y_test, lgbm_pred_rounded))
print(68 * '-')
print(confusion_matrix(y_test, lgbm_pred_rounded))
print(68 * '-')
fpr = ((y_test == 0) & (lgbm_pred_rounded == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (lgbm_pred_rounded == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

LGBM Benchmark Total Test
Accuracy Score: 0.976475
Precision Score: 0.9913886453875109
Recall Score: 0.9613
F1 Score: 0.9761125072982509
--------------------------------------------------------------------
[[99165   835]
 [ 3870 96130]]
--------------------------------------------------------------------
FPR:  0.00835
FNR:  0.0387
--------------------------------------------------------------------


In [146]:
lgbm_pred = lgbm_model.predict(X_test_dlls)

In [147]:
lgbm_pred_rounded = np.where(lgbm_pred >= 0.9, 1, 0)

In [149]:
print('LGBM DLL Test')
print("Accuracy Score:", accuracy_score(y_test_pd[mask], lgbm_pred_rounded))
print("Precision Score:", precision_score(y_test_pd[mask], lgbm_pred_rounded))
print("Recall Score:",recall_score(y_test_pd[mask], lgbm_pred_rounded))
print("F1 Score:",f1_score(y_test_pd[mask], lgbm_pred_rounded))
print(68 * '-')
print(confusion_matrix(y_test[mask], lgbm_pred_rounded))
print(68 * '-')
y_Series = pd.Series(np.array(y_test_pd[mask][0]))
fpr = ((y_Series == 0) & (lgbm_pred_rounded == 1)).sum() / (y_Series == 0).sum()
print("FPR: ", fpr)
fnr = ((y_Series == 1) & (lgbm_pred_rounded == 0)).sum() / (y_Series == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

LGBM DLL Test
Accuracy Score: 0.9645437516724645
Precision Score: 0.9895376307796152
Recall Score: 0.8888888888888888
F1 Score: 0.9365168090713087
--------------------------------------------------------------------
[[15763    62]
 [  733  5864]]
--------------------------------------------------------------------
FPR:  0.0039178515007898896
FNR:  0.1111111111111111
--------------------------------------------------------------------


## Sklearn Decision Tree Classifier

In [150]:
tr1 = load('tree_basic_25Apr.joblib')
tr1_pred = tr1.predict(X_test)
print(68 * '-')
print('TR1 Total Test')
print("Accuracy Score:", accuracy_score(y_test, tr1_pred))
print("Precision Score:", precision_score(y_test, tr1_pred))
print("Recall Score:",recall_score(y_test, tr1_pred))
print("F1 Score:",f1_score(y_test, tr1_pred))
print(68 * '-')
print(confusion_matrix(y_test, tr1_pred))
print(68 * '-')
fpr = ((y_test == 0) & (tr1_pred == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (tr1_pred == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
TR1 Total Test
Accuracy Score: 0.918685
Precision Score: 0.9202525419815913
Recall Score: 0.91682
F1 Score: 0.9185330641646671
--------------------------------------------------------------------
[[92055  7945]
 [ 8318 91682]]
--------------------------------------------------------------------
FPR:  0.07945
FNR:  0.08318
--------------------------------------------------------------------


In [151]:
tr1 = load('tree_basic_25Apr.joblib')
predX = X_test_pd[mask]
tr1_pred = tr1.predict(predX)
print(68 * '-')
print('TR1 DLL Only Test')
print("Accuracy Score:", accuracy_score(y_test_pd[mask], tr1_pred))
print("Precision Score:", precision_score(y_test_pd[mask], tr1_pred))
print("Recall Score:",recall_score(y_test_pd[mask], tr1_pred))
print("F1 Score:",f1_score(y_test_pd[mask], tr1_pred))
print(68 * '-')
print(confusion_matrix(y_test[mask], tr1_pred))
print(68 * '-')
y_Series = pd.Series(np.array(y_test_pd[mask][0]))
fpr = ((y_Series == 0) & (tr1_pred == 1)).sum() / (y_Series == 0).sum()
print("FPR: ", fpr)
fnr = ((y_Series == 1) & (tr1_pred == 0)).sum() / (y_Series == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
TR1 DLL Only Test
Accuracy Score: 0.9191419141914191
Precision Score: 0.8751568381430364
Recall Score: 0.8458390177353342
F1 Score: 0.8602482078162337
--------------------------------------------------------------------
[[15029   796]
 [ 1017  5580]]
--------------------------------------------------------------------
FPR:  0.0503001579778831
FNR:  0.15416098226466576
--------------------------------------------------------------------


## Sklearn Random Forrest Classifier

In [152]:
rf9 = load('RF_25AprNest25.joblib')
rf9_pred = rf9.predict(X_test)
print(68 * '-')
print('RF9 Basic Test')
print("Accuracy Score:", accuracy_score(y_test, rf9_pred))
print("Precision Score:", precision_score(y_test, rf9_pred))
print("Recall Score:",recall_score(y_test, rf9_pred))
print("F1 Score:",f1_score(y_test, rf9_pred))
print(68 * '-')
print(confusion_matrix(y_test, rf9_pred))
print(68 * '-')
fpr = ((y_test == 0) & (rf9_pred == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (rf9_pred == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
RF9 Basic Test
Accuracy Score: 0.953925
Precision Score: 0.9587000677048071
Recall Score: 0.94872
F1 Score: 0.9536839248287335
--------------------------------------------------------------------
[[95913  4087]
 [ 5128 94872]]
--------------------------------------------------------------------
FPR:  0.04087
FNR:  0.05128
--------------------------------------------------------------------


In [153]:
rf9 = load('RF_25AprNest25.joblib')
predX = X_test_pd[mask]
rf9_pred = rf9.predict(predX)
print('RF9 DLL Test')
print("Accuracy Score:", accuracy_score(y_test_pd[mask], rf9_pred))
print("Precision Score:", precision_score(y_test_pd[mask], rf9_pred))
print("Recall Score:",recall_score(y_test_pd[mask], rf9_pred))
print("F1 Score:",f1_score(y_test_pd[mask], rf9_pred))
print(68 * '-')
print(confusion_matrix(y_test[mask], rf9_pred))
print(68 * '-')
y_Series = pd.Series(np.array(y_test_pd[mask][0]))
fpr = ((y_Series == 0) & (rf9_pred == 1)).sum() / (y_Series == 0).sum()
print("FPR: ", fpr)
fnr = ((y_Series == 1) & (rf9_pred == 0)).sum() / (y_Series == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

RF9 DLL Test
Accuracy Score: 0.9442957809294443
Precision Score: 0.9456666666666667
Recall Score: 0.8600879187509474
F1 Score: 0.9008494085893467
--------------------------------------------------------------------
[[15499   326]
 [  923  5674]]
--------------------------------------------------------------------
FPR:  0.020600315955766193
FNR:  0.1399120812490526
--------------------------------------------------------------------


In [154]:
rf9 = load('RF9_DLLW.joblib')
rf9_pred = rf9.predict(X_test)
print(68 * '-')
print('RF9 DLL Tuned Total Test ')
print("Accuracy Score:", accuracy_score(y_test, rf9_pred))
print("Precision Score:", precision_score(y_test, rf9_pred))
print("Recall Score:",recall_score(y_test, rf9_pred))
print("F1 Score:",f1_score(y_test, rf9_pred))
print(68 * '-')
print(confusion_matrix(y_test, rf9_pred))
print(68 * '-')
fpr = ((y_test == 0) & (et1_pred == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (et1_pred == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
RF9 DLL Tuned Total Test 
Accuracy Score: 0.9519
Precision Score: 0.9565937841005537
Recall Score: 0.94676
F1 Score: 0.9516514886516696
--------------------------------------------------------------------
[[95704  4296]
 [ 5324 94676]]
--------------------------------------------------------------------
FPR:  0.03629
FNR:  0.05833
--------------------------------------------------------------------


In [155]:
rf9 = load('RF9_DLLW.joblib')
predX = X_test_pd[mask]
rf9_pred = rf9.predict(predX)
print('RF9 DLL Tuned DLL Only Test')
print("Accuracy Score:", accuracy_score(y_test_pd[mask], rf9_pred))
print("Precision Score:", precision_score(y_test_pd[mask], rf9_pred))
print("Recall Score:",recall_score(y_test_pd[mask], rf9_pred))
print("F1 Score:",f1_score(y_test_pd[mask], rf9_pred))
print(68 * '-')
print(confusion_matrix(y_test[mask], rf9_pred))
print(68 * '-')
y_Series = pd.Series(np.array(y_test_pd[mask][0]))
fpr = ((y_Series == 0) & (rf9_pred == 1)).sum() / (y_Series == 0).sum()
print("FPR: ", fpr)
fnr = ((y_Series == 1) & (rf9_pred == 0)).sum() / (y_Series == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

RF9 DLL Tuned DLL Only Test
Accuracy Score: 0.9523236107394524
Precision Score: 0.9492847854356307
Recall Score: 0.8852508716083068
F1 Score: 0.9161502862969645
--------------------------------------------------------------------
[[15513   312]
 [  757  5840]]
--------------------------------------------------------------------
FPR:  0.019715639810426542
FNR:  0.1147491283916932
--------------------------------------------------------------------


## Sklearn Extremely Random Forest Classifier

In [156]:
et1 = load('ET1.joblib')
et1_pred = et1.predict(X_test)
print(68 * '-')
print('ET1 Total Test')
print("Accuracy Score:", accuracy_score(y_test, et1_pred))
print("Precision Score:", precision_score(y_test, et1_pred))
print("Recall Score:",recall_score(y_test, et1_pred))
print("F1 Score:",f1_score(y_test, et1_pred))
print(68 * '-')
print(confusion_matrix(y_test, et1_pred))
print(68 * '-')
fpr = ((y_test == 0) & (et1_pred == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (et1_pred == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
ET1 Total Test
Accuracy Score: 0.954415
Precision Score: 0.9641197438437733
Recall Score: 0.94396
F1 Score: 0.9539333734190967
--------------------------------------------------------------------
[[96487  3513]
 [ 5604 94396]]
--------------------------------------------------------------------
FPR:  0.03513
FNR:  0.05604
--------------------------------------------------------------------


In [157]:
et1 = load('ET1.joblib')
predX = X_test_pd[mask]
et1_pred_dll = et1.predict(predX)
print('ET1 DLL Only Test')
print("Accuracy Score:", accuracy_score(y_test_pd[mask], et1_pred_dll))
print("Precision Score:", precision_score(y_test_pd[mask], et1_pred_dll))
print("Recall Score:",recall_score(y_test_pd[mask], et1_pred_dll))
print("F1 Score:",f1_score(y_test_pd[mask], et1_pred_dll))
print(68 * '-')
print(confusion_matrix(y_test[mask], rf9_pred))
print(68 * '-')
y_Series = pd.Series(np.array(y_test_pd[mask][0]))
fpr = ((y_Series == 0) & (et1_pred_dll == 1)).sum() / (y_Series == 0).sum()
print("FPR: ", fpr)
fnr = ((y_Series == 1) & (et1_pred_dll == 0)).sum() / (y_Series == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

ET1 DLL Only Test
Accuracy Score: 0.9417090357684417
Precision Score: 0.952755905511811
Recall Score: 0.843716840988328
F1 Score: 0.8949272449553822
--------------------------------------------------------------------
[[15513   312]
 [  757  5840]]
--------------------------------------------------------------------
FPR:  0.017440758293838864
FNR:  0.15628315901167197
--------------------------------------------------------------------


In [158]:
et1_dll = load('ET1_DLL.joblib')
et1_pred = et1_dll.predict(X_test)
print(68 * '-')
print('ET1 DLL Tuned Total Test')
print("Accuracy Score:", accuracy_score(y_test, et1_pred))
print("Precision Score:", precision_score(y_test, et1_pred))
print("Recall Score:",recall_score(y_test, et1_pred))
print("F1 Score:",f1_score(y_test, et1_pred))
print(68 * '-')
print(confusion_matrix(y_test, et1_pred))
print(68 * '-')
fpr = ((y_test == 0) & (et1_pred == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (et1_pred == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
ET1 DLL Tuned Total Test
Accuracy Score: 0.95269
Precision Score: 0.9628921428279276
Recall Score: 0.94167
F1 Score: 0.952162834435479
--------------------------------------------------------------------
[[96371  3629]
 [ 5833 94167]]
--------------------------------------------------------------------
FPR:  0.03629
FNR:  0.05833
--------------------------------------------------------------------


In [159]:
et1_dll = load('ET1_DLL.joblib')
predX = X_test_pd[mask]
et1_dll_pred = et1_dll.predict(predX)
print('ET1 DLL Tuned DLL Only Test')
print("Accuracy Score:", accuracy_score(y_test_pd[mask], et1_dll_pred))
print("Precision Score:", precision_score(y_test_pd[mask], et1_dll_pred))
print("Recall Score:",recall_score(y_test_pd[mask], et1_dll_pred))
print("F1 Score:",f1_score(y_test_pd[mask], et1_dll_pred))
print(68 * '-')
print(confusion_matrix(y_test[mask], et1_dll_pred))
print(68 * '-')
y_Series = pd.Series(np.array(y_test_pd[mask][0]))
fpr = ((y_Series == 0) & (et1_dll_pred == 1)).sum() / (y_Series == 0).sum()
print("FPR: ", fpr)
fnr = ((y_Series == 1) & (et1_dll_pred == 0)).sum() / (y_Series == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

ET1 DLL Tuned DLL Only Test
Accuracy Score: 0.9480867005619481
Precision Score: 0.9543401906673357
Recall Score: 0.8649386084583902
F1 Score: 0.9074427480916031
--------------------------------------------------------------------
[[15552   273]
 [  891  5706]]
--------------------------------------------------------------------
FPR:  0.01725118483412322
FNR:  0.13506139154160982
--------------------------------------------------------------------


## Sklearn Logistic Regression

In [160]:
LR1 = load('LR1.joblib')
LR1_pred = LR1.predict(X_test)
print(68 * '-')
print('LR1 DLL Tuned Total Test')
print("Accuracy Score:", accuracy_score(y_test, LR1_pred))
print("Precision Score:", precision_score(y_test, LR1_pred))
print("Recall Score:",recall_score(y_test, LR1_pred))
print("F1 Score:",f1_score(y_test, LR1_pred))
print(68 * '-')
print(confusion_matrix(y_test, LR1_pred))
print(68 * '-')
fpr = ((y_test == 0) & (LR1_pred == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (LR1_pred == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
LR1 DLL Tuned Total Test
Accuracy Score: 0.62176
Precision Score: 0.5898756975405238
Recall Score: 0.79914
F1 Score: 0.6787443306324211
--------------------------------------------------------------------
[[44438 55562]
 [20086 79914]]
--------------------------------------------------------------------
FPR:  0.55562
FNR:  0.20086
--------------------------------------------------------------------


In [161]:
LR1_dll = load('LR1.joblib')
predX = X_test_pd[mask]
LR1_dll_pred = LR1_dll.predict(predX)
print('LR1 DLL Tuned DLL Only Test')
print("Accuracy Score:", accuracy_score(y_test_pd[mask], LR1_dll_pred))
print("Precision Score:", precision_score(y_test_pd[mask], LR1_dll_pred))
print("Recall Score:",recall_score(y_test_pd[mask], LR1_dll_pred))
print("F1 Score:",f1_score(y_test_pd[mask], LR1_dll_pred))
print(68 * '-')
print(confusion_matrix(y_test[mask], LR1_dll_pred))
print(68 * '-')
y_Series = pd.Series(np.array(y_test_pd[mask][0]))
fpr = ((y_Series == 0) & (LR1_dll_pred == 1)).sum() / (y_Series == 0).sum()
print("FPR: ", fpr)
fnr = ((y_Series == 1) & (LR1_dll_pred == 0)).sum() / (y_Series == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

LR1 DLL Tuned DLL Only Test
Accuracy Score: 0.5121755418785122
Precision Score: 0.3288114204590267
Recall Score: 0.6319539184477793
F1 Score: 0.43255862212077195
--------------------------------------------------------------------
[[7315 8510]
 [2428 4169]]
--------------------------------------------------------------------
FPR:  0.5377567140600316
FNR:  0.3680460815522207
--------------------------------------------------------------------


## Sklearn Gradient Boosting Classifer

In [5]:
gbc = load('gbc1_rf9.joblib')
gbc_pred = gbc.predict(X_test)
print(68 * '-')
print('gbc DLL Tuned Total Test')
print("Accuracy Score:", accuracy_score(y_test, gbc_pred))
print("Precision Score:", precision_score(y_test, gbc_pred))
print("Recall Score:",recall_score(y_test, gbc_pred))
print("F1 Score:",f1_score(y_test, gbc_pred))
print(68 * '-')
print(confusion_matrix(y_test, gbc_pred))
print(68 * '-')
fpr = ((y_test == 0) & (gbc_pred == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (gbc_pred == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
gbc DLL Tuned Total Test
Accuracy Score: 0.901675
Precision Score: 0.8847130036682661
Recall Score: 0.92372
F1 Score: 0.903795821123336
--------------------------------------------------------------------
[[87963 12037]
 [ 7628 92372]]
--------------------------------------------------------------------
FPR:  0.12037
FNR:  0.07628
--------------------------------------------------------------------


In [6]:
gbc = load('gbc1_rf9.joblib')
predX = X_test_pd[mask]
gbc_dll_pred = gbc.predict(predX)
print('gbc DLL Tuned DLL Only Test')
print("Accuracy Score:", accuracy_score(y_test_pd[mask], gbc_dll_pred))
print("Precision Score:", precision_score(y_test_pd[mask], gbc_dll_pred))
print("Recall Score:",recall_score(y_test_pd[mask], gbc_dll_pred))
print("F1 Score:",f1_score(y_test_pd[mask], gbc_dll_pred))
print(68 * '-')
print(confusion_matrix(y_test[mask], gbc_dll_pred))
print(68 * '-')
y_Series = pd.Series(np.array(y_test_pd[mask][0]))
fpr = ((y_Series == 0) & (gbc_dll_pred == 1)).sum() / (y_Series == 0).sum()
print("FPR: ", fpr)
fnr = ((y_Series == 1) & (gbc_dll_pred == 0)).sum() / (y_Series == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

gbc DLL Tuned DLL Only Test
Accuracy Score: 0.8841762554633842
Precision Score: 0.810077519379845
Recall Score: 0.792026678793391
F1 Score: 0.8009504100559516
--------------------------------------------------------------------
[[14600  1225]
 [ 1372  5225]]
--------------------------------------------------------------------
FPR:  0.07740916271721959
FNR:  0.20797332120660905
--------------------------------------------------------------------


In [18]:
gbc_prob = gbc.predict_proba(X_test)

In [25]:
gbc_prob_rounded = np.where(gbc_prob[:, 1] >= 0.5, 1, 0)

In [26]:
print("Accuracy Score:", accuracy_score(y_test, gbc_prob_rounded))
print("Precision Score:", precision_score(y_test, gbc_prob_rounded))
print("Recall Score:",recall_score(y_test, gbc_prob_rounded))
print("F1 Score:",f1_score(y_test, gbc_prob_rounded))
print(68 * '-')
print(confusion_matrix(y_test, gbc_prob_rounded))
print(68 * '-')
fpr = ((y_test == 0) & (gbc_prob_rounded == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (gbc_prob_rounded == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

Accuracy Score: 0.901675
Precision Score: 0.8847130036682661
Recall Score: 0.92372
F1 Score: 0.903795821123336
--------------------------------------------------------------------
[[87963 12037]
 [ 7628 92372]]
--------------------------------------------------------------------
FPR:  0.12037
FNR:  0.07628
--------------------------------------------------------------------


In [ ]:
--------------------------------------------------------------------
gbc DLL Tuned Total Test
Accuracy Score: 0.901675
Precision Score: 0.8847130036682661
Recall Score: 0.92372
F1 Score: 0.903795821123336
--------------------------------------------------------------------
[[87963 12037]
 [ 7628 92372]]
--------------------------------------------------------------------
FPR:  0.12037
FNR:  0.07628

## Sklearn AdaBoost Classifier with a Random Forest Classifer

In [167]:
adaboost_clf = load('ADB_RF9.joblib')
AB_pred = adaboost_clf.predict(X_test)
print(68 * '-')
print('AdaBoost DLL Tuned Total Test')
print("Accuracy Score:", accuracy_score(y_test, AB_pred))
print("Precision Score:", precision_score(y_test, AB_pred))
print("Recall Score:",recall_score(y_test, AB_pred))
print("F1 Score:",f1_score(y_test, AB_pred))
print(68 * '-')
print(confusion_matrix(y_test, AB_pred))
print(68 * '-')
fpr = ((y_test == 0) & (AB_pred == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (AB_pred == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
AdaBoost DLL Tuned Total Test
Accuracy Score: 0.95125
Precision Score: 0.9584662589153272
Recall Score: 0.94338
F1 Score: 0.9508632941247619
--------------------------------------------------------------------
[[95912  4088]
 [ 5662 94338]]
--------------------------------------------------------------------
FPR:  0.04088
FNR:  0.05662
--------------------------------------------------------------------


In [169]:
adaboost_clf = load('ADB_RF9.joblib')
predX = X_test_pd[mask]
AB_dll_pred = adaboost_clf.predict(predX)
print('AdaBoost DLL Tuned DLL Only Test')
print("Accuracy Score:", accuracy_score(y_test_pd[mask], AB_dll_pred))
print("Precision Score:", precision_score(y_test_pd[mask], AB_dll_pred))
print("Recall Score:",recall_score(y_test_pd[mask], AB_dll_pred))
print("F1 Score:",f1_score(y_test_pd[mask], AB_dll_pred))
print(68 * '-')
print(confusion_matrix(y_test[mask], AB_dll_pred))
print(68 * '-')
y_Series = pd.Series(np.array(y_test_pd[mask][0]))
fpr = ((y_Series == 0) & (AB_dll_pred == 1)).sum() / (y_Series == 0).sum()
print("FPR: ", fpr)
fnr = ((y_Series == 1) & (AB_dll_pred == 0)).sum() / (y_Series == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

AdaBoost DLL Tuned DLL Only Test
Accuracy Score: 0.9466595308179466
Precision Score: 0.9464374276739957
Recall Score: 0.8678187054721843
F1 Score: 0.9054246402024355
--------------------------------------------------------------------
[[15501   324]
 [  872  5725]]
--------------------------------------------------------------------
FPR:  0.020473933649289098
FNR:  0.13218129452781568
--------------------------------------------------------------------


# Neural Network Models 

## Tensor Flow Keras Squential with 8 Dense layers, adam optimizer, and 128 epochs

In [34]:
model_seq = load('SeqNN_8D_128E.joblib') 

In [39]:
model_seq.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 64)             │       152,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 478,205 (1.82 MB)

 Trainable params: 159,401 (622.66 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 318,804 (1.22 MB)

In [37]:
seq_pred_per = model_seq.predict(X_test_pd)

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step


In [38]:
seq_pred_per = np.where(seq_pred_per >= 0.5, 1, 0)

In [170]:
print(68 * '-')
print('Seq Basic Test')
print("Accuracy Score:", accuracy_score(y_test, seq_pred_per))
print("Precision Score:", precision_score(y_test, seq_pred_per))
print("Recall Score:",recall_score(y_test, seq_pred_per))
print("F1 Score:",f1_score(y_test, seq_pred_per))
print(68 * '-')
print(confusion_matrix(y_test, seq_pred_per))
print(68 * '-')
fpr = ((y_test == 0) & (seq_pred_per.flatten() == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (seq_pred_per.flatten() == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
Seq Basic Test
Accuracy Score: 0.652775
Precision Score: 0.7803879824545304
Recall Score: 0.42521
F1 Score: 0.5504799756613825
--------------------------------------------------------------------
[[88034 11966]
 [57479 42521]]
--------------------------------------------------------------------
FPR:  0.11966
FNR:  0.57479
--------------------------------------------------------------------


## Tensor Flow Keras Squential with 9 Dense layers, adam optimizer, and 32 epochs

In [42]:
model_seq2 = load('SeqNN_9D_32E.joblib') 

In [43]:
model_seq2.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_33 (Dense)                │ (None, 256)            │       609,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,961,213 (7.48 MB)

 Trainable params: 653,737 (2.49 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,307,476 (4.99 MB)

In [52]:
seq_pred_per2 = model_seq2.predict(X_test_pd)

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 48s 8ms/step


In [79]:
seq_pred_rounded2.flatten()

array([1, 1, 1, ..., 0, 1, 1])

In [57]:
seq_pred_rounded2 = np.where(seq_pred_per2 >= 0.45, 1, 0)

In [171]:
print(68 * '-')
print('Seq2 Basic Test')
print("Accuracy Score:", accuracy_score(y_test, seq_pred_rounded2))
print("Precision Score:", precision_score(y_test, seq_pred_rounded2))
print("Recall Score:",recall_score(y_test, seq_pred_rounded2))
print("F1 Score:",f1_score(y_test, seq_pred_rounded2))
print(68 * '-')
print(confusion_matrix(y_test, seq_pred_rounded2))
print(68 * '-')
fpr = ((y_test == 0) & (seq_pred_rounded2.flatten() == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (seq_pred_rounded2.flatten() == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
Seq2 Basic Test
Accuracy Score: 0.60608
Precision Score: 0.5749417167078771
Recall Score: 0.81383
F1 Score: 0.6738397847236597
--------------------------------------------------------------------
[[39833 60167]
 [18617 81383]]
--------------------------------------------------------------------
FPR:  0.60167
FNR:  0.18617
--------------------------------------------------------------------


## Tensor Flow Convolutional Neural Network

In [59]:
model_conv = load('ConvNN.joblib') 

In [60]:
model_conv.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 2374, 64)       │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 151936)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 1)              │       151,937 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 457,541 (1.75 MB)

 Trainable params: 152,513 (595.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 305,028 (1.16 MB)

In [61]:
model_conv_pred = model_conv.predict(X_test_pd)

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 36s 6ms/step


In [62]:
model_conv_pred_rounded = np.where(model_conv_pred >= 0.5, 1, 0)

In [172]:
print(68 * '-')
print('Seq Basic Test')
print("Accuracy Score:", accuracy_score(y_test, model_conv_pred_rounded))
print("Precision Score:", precision_score(y_test, model_conv_pred_rounded))
print("Recall Score:",recall_score(y_test, model_conv_pred_rounded))
print("F1 Score:",f1_score(y_test, model_conv_pred_rounded))
print(68 * '-')
print(confusion_matrix(y_test, model_conv_pred_rounded))
print(68 * '-')
fpr = ((y_test == 0) & (model_conv_pred_rounded.flatten() == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (model_conv_pred_rounded.flatten() == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
Seq Basic Test
Accuracy Score: 0.64167
Precision Score: 0.6191846280685814
Recall Score: 0.736
F1 Score: 0.672557638006817
--------------------------------------------------------------------
[[54734 45266]
 [26400 73600]]
--------------------------------------------------------------------
FPR:  0.45266
FNR:  0.264
--------------------------------------------------------------------


## Tensor Flow Simple Recurrent Neural Network

In [5]:
model_rnn = load('SimpleRNN_1.joblib') 

In [6]:
model_rnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 64)             │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,869 (50.27 KB)

 Trainable params: 4,289 (16.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,580 (33.52 KB)

In [8]:
model_rnn_pred = model_rnn.predict(X_test_pd)

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 300s 48ms/step


In [9]:
model_rnn_pred_rounded = np.where(model_rnn_pred >= 0.5, 1, 0)

In [11]:
print(68 * '-')
print('Seq Basic Test')
print("Accuracy Score:", accuracy_score(y_test, model_rnn_pred_rounded))
print("Precision Score:", precision_score(y_test, model_rnn_pred_rounded))
print("Recall Score:",recall_score(y_test, model_rnn_pred_rounded))
print("F1 Score:",f1_score(y_test, model_rnn_pred_rounded))
print(68 * '-')
print(confusion_matrix(y_test, model_rnn_pred_rounded))
print(68 * '-')
fpr = ((y_test == 0) & (model_rnn_pred_rounded.flatten() == 1)).sum() / (y_test == 0).sum()
print("FPR: ", fpr)
fnr = ((y_test == 1) & (model_rnn_pred_rounded.flatten() == 0)).sum() / (y_test == 1).sum()
print("FNR: ", fnr)
print(68 * '-')

--------------------------------------------------------------------
Seq Basic Test
Accuracy Score: 0.638275
Precision Score: 0.5874333697968371
Recall Score: 0.92902
F1 Score: 0.7197548702493521
--------------------------------------------------------------------
[[34753 65247]
 [ 7098 92902]]
--------------------------------------------------------------------
FPR:  0.65247
FNR:  0.07098
--------------------------------------------------------------------
